In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
filenames = os.listdir("../text")

In [4]:
from jaconv import normalize

episode = pd.read_csv("../ling_summary_processed.csv")
episode = episode.set_index("Id")
episode["Title"] = episode["Title"].apply(normalize)
episode

,Title,ThumbnailUrl,VideoUrl,PublicationDate,IsAnalyzed,Channel
Id,,,,,,
1,「イルカも喋る」は大ウソ【言語学って何?】#1,https://i.ytimg.com/vi/2YY9DT4uDh0/sddefault.jpg,https://www.youtube.com/watch?v=2YY9DT4uDh0,2021-03-11,True,ゆる言語学ラジオ
2,2km先では言語が違う国があるらしい...【言語がたくさんある理由】#2,https://i.ytimg.com/vi/-Zo_0_DZrvk/sddefault.jpg,https://www.youtube.com/watch?v=-Zo_0_DZrvk,2021-03-13,True,ゆる言語学ラジオ
3,藤原不比等は「ぷぢぃぱらのぷぴちょ」だった #3,https://i.ytimg.com/vi/KItCvPD86pw/sddefault.jpg,https://www.youtube.com/watch?v=KItCvPD86pw,2021-03-15,True,ゆる言語学ラジオ
4,悶・聞・関、部首が「門」なのはどれ? #4,https://i.ytimg.com/vi/v2vY-H1FAHM/sddefault.jpg,https://www.youtube.com/watch?v=v2vY-H1FAHM,2021-03-16,True,ゆる言語学ラジオ
5,英語は荒野行動!?日本語に「時制の一致」が要らない理由 #5,https://i.ytimg.com/vi/UEc3nobDjMk/sddefault.jpg,https://www.youtube.com/watch?v=UEc3nobDjMk,2021-03-17,True,ゆる言語学ラジオ
...,...,...,...,...,...,...
169,すべての言語の共通点とは?【生成文法1】#167,https://i.ytimg.com/vi/E49cMz_QwO8/sddefault.jpg,https://www.youtube.com/watch?v=E49cMz_QwO8,2022-10-11,True,ゆる言語学ラジオ
170,神になったら、人間に何をインストールすべき?【生成文法2】#168,https://i.ytimg.com/vi/_xvgxuvfcts/sddefault.jpg,https://www.youtube.com/watch?v=_xvgxuvfcts,2022-10-14,True,ゆる言語学ラジオ
171,チョムスキーが提唱した「ふたまたニョキニョキ理論」【生成文法3】#169,https://i.ytimg.com/vi/CYxGKxBZApE/sddefault.jpg,https://www.youtube.com/watch?v=CYxGKxBZApE,2022-10-18,True,ゆる言語学ラジオ


In [46]:
episode.loc[87]

title                                ゆる言語学ラジオ忘年会ライブ【流行語大賞決定&ひたすらエモい話】
thumbnail_url       https://i.ytimg.com/vi/poT4BzX7e_Q/sddefault.j...
video_url                 https://www.youtube.com/watch?v=poT4BzX7e_Q
publication_date                                           2021-12-28
Name: 87, dtype: object

'/Users/matsukokuumahikari/ghq/github.com/wakamenori/yuru-corpus-backend/data'

In [22]:
from jaconv import normalize

In [7]:
from tqdm import tqdm

filenames = os.listdir("../text")
# def get_text_filepath(filename: str):
#     cwd = os.getcwd()
#     parent = "/".join(os.getcwd().split("/")[:-1])
#     return os.path.join(parent, filename)

def convert_1d_to_2d(l, cols):
    return [l[i : i + cols] for i in range(0, len(l), cols)]


def read_file(filename: str):
    with open("../text/" + filename, "r") as f:
        lines = f.readlines()[2:-1]
    lines = [line.replace("\n", "") for line in lines if line != "\n"]
    lines_2d = convert_1d_to_2d(lines, 2)
    return lines_2d


def extract_info(line):
    timestamp = line[0].split(".")[0]
    if len(timestamp) == 5:
        timestamp = "00:" + timestamp
    text = line[1]
    # for p in parentheses.itertuples():
    #     text = text.replace(p.text, "")
    # for s in spaces.itertuples():
    #     if text.startswith(s.text +" "):
    #         text = text[len(s)+1:]
    #         break
    # text = text.replace("【", "").replace("】", "")
    return timestamp, text


def get_episode_id(title: str):
    matched = episode[episode["Title"] == normalize(title)]
    if len(matched) == 0:
        raise ValueError(title)
    return matched.index[0]


morphemes = []

for filename in tqdm(filenames):
    if not filename.endswith("vtt"):
        continue
    episode_id = get_episode_id(".".join(filename.split(".")[:-1]))
    # print(filename, episode_id)
    lines = read_file(filename)
    for line in lines:
        timestamp, text = extract_info(line)
        record = {
            "Type": "Morpheme",
            "Id": f"{episode_id}#{timestamp}#0",
            "Token": text,
            "Speaker": ""
        }
        morphemes.append(record)

morphemes_df = pd.DataFrame(morphemes)
morphemes_df = morphemes_df.drop_duplicates(subset=["Id"])
morphemes_df.to_csv("../morphemes.csv", index=False)
morphemes_df

100%|███████████████████████████████████████| 324/324 [00:02<00:00, 131.29it/s]


,Type,Id,Token,Speaker
0,Morpheme,75#00:00:00#0,ゆる言語学ラジオ,
1,Morpheme,75#00:00:31#0,ほりまさん、ちなみに第二言語習得論っていうと,
2,Morpheme,75#00:00:34#0,どういうようなイメージ,
3,Morpheme,75#00:00:36#0,もう本当にマジでイメージでいいんで,
4,Morpheme,75#00:00:38#0,どんなことやってると思いますか?,
...,...,...,...,...
155891,Morpheme,33#00:44:21#0,ちょっと外国語勉強してみようという気に,
155892,Morpheme,33#00:44:23#0,あったんじゃないでしょうか皆さん,
155893,Morpheme,33#00:44:25#0,そんな感じですかね今回は,
155894,Morpheme,33#00:44:27#0,ということで今回は,


In [17]:
from tqdm import tqdm
spaces = pd.read_csv("../space.csv")
parentheses = pd.read_csv("../【】.csv")

morphemes_df = pd.read_csv("../morphemes.csv")

for space in tqdm(spaces.itertuples()):
    def retrieve(token: str):
        if token.startswith(space.text):
            return token[len(space)+1:]
        return token
    
    morphemes_df["Token"] = np.vectorize(retrieve)(morphemes_df["Token"])


for space in tqdm(parentheses.itertuples()):
    def retrieve(token: str):
        return token.replace(space.text, "")

    morphemes_df["Token"] = np.vectorize(retrieve)(morphemes_df["Token"])


def retrieve(token: str):
        return token.replace("】", "").replace("【", "")


morphemes_df["Token"] = np.vectorize(retrieve)(morphemes_df["Token"])
morphemes_df = morphemes_df[morphemes_df["Token"] != ""]
morphemes_df["Speaker"] = ""
morphemes_df = morphemes_df.sort_values("Id")
morphemes_df

11it [00:05,  1.94it/s]
46it [00:17,  2.64it/s]


,Type,Id,Token,Speaker
0,Morpheme,75#00:00:00#0,ゆる言語学ラジオ,NaN
1,Morpheme,75#00:00:31#0,ほりまさん、ちなみに第二言語習得論っていうと,NaN
2,Morpheme,75#00:00:34#0,どういうようなイメージ,NaN
3,Morpheme,75#00:00:36#0,もう本当にマジでイメージでいいんで,NaN
4,Morpheme,75#00:00:38#0,どんなことやってると思いますか?,NaN
...,...,...,...,...
155373,Morpheme,33#00:44:21#0,ちょっと外国語勉強してみようという気に,NaN
155374,Morpheme,33#00:44:23#0,あったんじゃないでしょうか皆さん,NaN
155375,Morpheme,33#00:44:25#0,そんな感じですかね今回は,NaN
155376,Morpheme,33#00:44:27#0,ということで今回は,NaN


In [19]:
morphemes_df["Speaker"] = ""
morphemes_df = morphemes_df.sort_values("Id")
morphemes_df.to_csv("../morphemes_processed.csv", index=False)

In [20]:
morphemes_df[morphemes_df["Id"].str.startswith("1#")].to_csv("../test/morphemes_processed.csv", index=False)

In [13]:
morphemes_df = morphemes_df[morphemes_df["Token"] == ""]

In [12]:
err

85712    久しぶりですね
Name: Token, dtype: object

In [19]:
all = []
for row in morphemes_df["Token"].str.findall("【.*?】"):
    all.extend([morpheme for morpheme in row])
all = pd.DataFrame(all)
count = all.value_counts()
count = count[count != 1]
count.to_csv("../【】_result.csv")
count

Series([], dtype: int64)

In [20]:
space = morphemes_df["Token"].str.split(" ", expand=True)[0].value_counts()
space = space[space > 99]
space.to_csv("../space_result.csv")
space

               6412
そうですね           829
しゃちょー           752
はい              660
しゃちょーエスペラント     573
しゃちょーエンディング     526
語学ラジオ           400
うん              348
あのー             337
なるほど            329
質疑応答            278
はいはい            272
そうだね            271
正解              229
へー              206
そうそうそう          197
この文章の中で         178
はいはいはい          154
なるほどなるほど        142
お               133
ありがとうございました     129
そうなんですよ         126
なるほどね           122
そうなんだ           109
ありがとうございます      108
Name: 0, dtype: int64

In [159]:
morphemes_df[morphemes_df["Token"].str.startswith("学習")].to_csv("学習.csv")

In [74]:
import os

import pandas as pd
from dotenv import load_dotenv

load_dotenv()

import boto3

db = boto3.resource(
    "dynamodb",
    # region_name="ap-northeast-1",
    endpoint_url="http://dynamodb-local:8000",
)

table = db.Table("YuruCorpusTest")


df = pd.read_csv("data/morphemes.csv")

from dataclasses import asdict, dataclass


@dataclass
class Item:
    Type: str
    Id: str
    Token: str
    Speaker: str


for row in df.itertuples():
    item = Item(Type="morpheme", Id=str(row.Id), Token=row.Token, Speaker="")
    table.put_item(Item=asdict(item))

In [14]:
df = pd.read_csv("../morphemes_processed.csv")
dev_df = df[df["Id"].str.startswith("1#")]
dev_df.to_csv("../dev/morphemes_processed.csv")